In [1]:
# union the last few years worth of games together then perform feauture engineering

In [2]:
# imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import numpy as np
import pandas as pd


In [3]:
# get games data for last few years
games_0 = pd.read_excel('data/nhl_game_results_2023.xlsx', header=0)
games_1 = pd.read_excel('data/nhl_game_results_2024.xlsx', header=0)
games_2 = pd.read_excel('data/nhl_game_results_2025.xlsx', header=0)

# union together
all_games = pd.concat([games_0, games_1, games_2], ignore_index=True)

# reset index
all_games.reset_index(drop=True, inplace=True)

# create Game_ID column
all_games["Game_ID"] = (
    all_games["Date"].dt.strftime("%Y-%m-%d") + "-" +
    all_games["Time"].astype(str) + "|" +
    all_games["Home Team"].astype(str) + " vs " +
    all_games["Away Team"].astype(str)
)

# inspect
all_games.info()
all_games.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Game Link      3349 non-null   object        
 1   Extra Time     3349 non-null   object        
 2   Home Team      3349 non-null   object        
 3   Away Team      3349 non-null   object        
 4   Home Score     3349 non-null   int64         
 5   Away Score     3349 non-null   int64         
 6   P1 Home Score  3349 non-null   int64         
 7   P1 Away Score  3349 non-null   int64         
 8   P2 Home Score  3349 non-null   int64         
 9   P2 Away Score  3349 non-null   int64         
 10  P3 Home Score  3349 non-null   int64         
 11  P3 Away Score  3349 non-null   int64         
 12  P4 Home Score  3349 non-null   int64         
 13  P4 Away Score  3349 non-null   int64         
 14  P5 Home Score  3349 non-null   int64         
 15  P5 Away Score  3349 n

,Game Link,Extra Time,Home Team,Away Team,Home Score,Away Score,P1 Home Score,P1 Away Score,P2 Home Score,P2 Away Score,...,P5 Away Score,P6 Home Score,P6 Away Score,P7 Home Score,P7 Away Score,Date,Time,Year,Total Score,Game_ID
0,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,2,1,1,1,1,0,...,0,0,0,0,0,2023-06-24,20:00,2023,3,2023-06-24-20:00|Florida Panthers vs Edmonton ...
1,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,5,1,1,0,2,0,...,0,0,0,0,0,2023-06-21,20:00,2023,6,2023-06-21-20:00|Edmonton Oilers vs Florida Pa...
2,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,3,5,0,1,2,3,...,0,0,0,0,0,2023-06-18,20:00,2023,8,2023-06-18-20:00|Florida Panthers vs Edmonton ...
3,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,8,1,3,1,3,0,...,0,0,0,0,0,2023-06-15,20:00,2023,9,2023-06-15-20:00|Edmonton Oilers vs Florida Pa...
4,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,3,4,0,1,1,3,...,0,0,0,0,0,2023-06-13,20:00,2023,7,2023-06-13-20:00|Edmonton Oilers vs Florida Pa...


In [4]:
# inspect basic stats for numeric cols
all_games.describe()

,Home Score,Away Score,P1 Home Score,P1 Away Score,P2 Home Score,P2 Away Score,P3 Home Score,P3 Away Score,P4 Home Score,P4 Away Score,P5 Home Score,P5 Away Score,P6 Home Score,P6 Away Score,P7 Home Score,P7 Away Score,Date,Year,Total Score
count,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.000000,3349.0,3349.0,3349.0,3349.0,3349,3349.000000,3349.000000
mean,3.210511,2.916990,0.919678,0.830397,1.074351,0.961780,1.103613,1.026874,0.081815,0.069573,0.068976,0.063302,0.0,0.0,0.0,0.0,2024-03-05 00:39:59.283368192,2023.649149,6.127501
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2023-01-01 00:00:00,2023.000000,1.000000
25%,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2023-09-24 00:00:00,2023.000000,5.000000
50%,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2024-01-27 00:00:00,2024.000000,6.000000
75%,4.000000,4.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2024-10-12 00:00:00,2024.000000,7.000000
max,10.000000,10.000000,5.000000,6.000000,7.000000,6.000000,7.000000,6.000000,1.000000,1.000000,4.000000,4.000000,0.0,0.0,0.0,0.0,2025-11-08 00:00:00,2025.000000,17.000000
std,1.757098,1.715671,0.954677,0.903608,1.017541,0.971073,1.056788,1.033429,0.274124,0.254464,0.351251,0.344170,0.0,0.0,0.0,0.0,NaN,0.654629,2.297768


In [5]:
# isolate the game links
game_urls = all_games['Game Link']

# store len of game_urls
len_games = len(game_urls)

# inspect first url
game_urls[0]

'https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/?mid=ro9gcXlC'

In [10]:
from urllib.parse import urlsplit, urlunsplit

# build odds urls from game url
odds_urls = []

for x in game_urls:
    # split url into parts
    parts = urlsplit(x)
    path = parts.path.rstrip("/")  # remove trailing slash

    # insert odds/1x2-odds/full-time/ before the last segment
    new_path = path + "/odds/1x2-odds/full-time/"

    # reconstruct URL with query string preserved (?mid=...)
    final_url = urlunsplit((parts.scheme, parts.netloc, new_path, parts.query, parts.fragment))

    # add to list
    odds_urls.append(final_url)

# inspect final odds urls
print(len(odds_urls))
odds_urls[:5]


3349


['https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/odds/1x2-odds/full-time/?mid=ro9gcXlC',
 'https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/odds/1x2-odds/full-time/?mid=h4AMJeNH',
 'https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/odds/1x2-odds/full-time/?mid=Qo6jnIyh',
 'https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/odds/1x2-odds/full-time/?mid=lhmVlZPk',
 'https://www.flashscore.com/match/hockey/edmonton-oilers-Mg0qoxeI/florida-panthers-fc1eq8Pp/odds/1x2-odds/full-time/?mid=I9dafe2S']

In [ ]:
# directly access the odds pages and scrape the 1x2 odds
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
import numpy as np
import pandas as pd

# ---------------------------
# Chrome speed-oriented setup
# ---------------------------
opts = Options()
opts.add_argument("--headless=new")
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-extensions")
opts.add_argument("--disable-notifications")
opts.add_argument("--blink-settings=imagesEnabled=false")  # block images

# also block heavy content types
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.fonts": 2
}
opts.add_experimental_option("prefs", prefs)
opts.page_load_strategy = "eager"  # don't wait for all subresources

driver = webdriver.Chrome(options=opts)
wait = WebDriverWait(driver, 8)

# define base url
base_url = 'https://www.flashscore.com/'

master_odds_1x2 = []

for idx, url in enumerate(odds_urls):
    # print every 100 urls
    if idx % 100 == 0:
        print(f'{idx} of {len_games}')

    try:    
        # navigate to url
        driver.get(url)

        # Wait until at least one ODD_CELL appears
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "[data-analytics-element^='ODDS_COMP'][data-analytics-element*='ODD_CELL_']")
            )
        )

        # ✅ Get ALL elements for cells ending in ODD_CELL_1, 2, and 3
        cell1_elems = driver.find_elements(
            By.CSS_SELECTOR, "[data-analytics-element^='ODDS_COMP'][data-analytics-element$='ODD_CELL_1'] span"
        )
        cell2_elems = driver.find_elements(
            By.CSS_SELECTOR, "[data-analytics-element^='ODDS_COMP'][data-analytics-element$='ODD_CELL_2'] span"
        )
        cell3_elems = driver.find_elements(
            By.CSS_SELECTOR, "[data-analytics-element^='ODDS_COMP'][data-analytics-element$='ODD_CELL_3'] span"
        )

        # get all the vals and replace + with ''
        cell1_vals = [float(e.text.replace("+", "")) for e in cell1_elems if e.text.strip()]
        cell2_vals = [float(e.text.replace("+", "")) for e in cell2_elems if e.text.strip()]
        cell3_vals = [float(e.text.replace("+", "")) for e in cell3_elems if e.text.strip()]

        # get means of values
        cell1_mean = np.mean(cell1_vals) if cell1_vals else np.nan
        cell2_mean = np.mean(cell2_vals) if cell2_vals else np.nan
        cell3_mean = np.mean(cell3_vals) if cell3_vals else np.nan

    except:
        print("Could not find odds for URL:", url)
        cell1_mean = np.nan
        cell2_mean = np.nan
        cell3_mean = np.nan

    # print(cell1_mean, cell2_mean, cell3_mean)

    # store final means in master list
    master_odds_1x2.append([cell1_mean, cell2_mean, cell3_mean])

driver.quit()

print("\n✅ FINAL RESULT (master_odds_1x2):")
print(master_odds_1x2)

0 of 3349
[2.45] [4.0] [2.45]
[2.35] [4.0] [2.55]
[2.0] [4.0] [3.1]
[2.3] [4.0] [2.55]
[2.15] [4.0] [2.8]
[2.2] [4.0] [2.75]
[2.15] [4.0] [2.8]
[2.2] [4.2] [2.75]
[1.95] [4.0] [3.3]
[2.4] [3.8] [2.6]
[2.7] [3.8] [2.3]
[2.2] [4.0] [2.75]
[1.95] [4.0] [3.25]
[2.25] [4.0] [2.7]
[2.1] [3.8] [3.1]
[2.15] [4.2] [2.75]
[2.55] [3.8] [2.45]
[2.3] [4.0] [2.65]
[2.5] [3.8] [2.4]
[3.0] [4.0] [2.05]
[1.8] [4.5] [3.4]
[2.35] [4.2] [2.45]
[2.8] [4.2] [2.05]
[2.8] [4.2] [2.1]
[2.0] [4.25] [2.9]
[2.15] [4.0] [2.8]
[1.83] [4.25] [3.5]
[1.8] [4.2] [3.75]
[2.15] [4.2] [2.75]
[2.5] [3.8] [2.5]
[1.87] [4.25] [3.4]
[3.2] [4.0] [2.0]
[2.2] [4.2] [2.7]
[2.15] [3.8] [2.9]
[2.8] [4.2] [2.15]
[2.7] [3.8] [2.3]
[2.2] [4.25] [2.65]
[2.05] [4.0] [3.1]
[2.75] [4.0] [2.2]
[1.95] [4.0] [3.3]
[2.35] [4.2] [2.5]
[2.6] [3.8] [2.4]
[1.91] [4.2] [3.25]
[2.15] [3.8] [3.0]
[2.5] [4.0] [2.4]
Could not find odds for URL: https://www.flashscore.com/match/hockey/boston-bruins-jeans23J/toronto-maple-leafs-ShE7Z8G3/odds/1x2-odds/fu

In [ ]:
# Convert master list to DataFrame
df_odds = pd.DataFrame(master_odds_1x2, columns=['Odds_1', 'Odds_X', 'Odds_2'])

# Concatenate side by side for final results
# final_data = pd.concat([all_games.iloc[:10, :], df_odds], axis=1) # test line to usse on test sample
final_data = pd.concat([all_games, df_odds], axis=1) 
final_data

,Game Link,Extra Time,Home Team,Away Team,Home Score,Away Score,P1 Home Score,P1 Away Score,P2 Home Score,P2 Away Score,...,P7 Home Score,P7 Away Score,Date,Time,Year,Total Score,Game_ID,1_Odds,X_Odds,2_Odds
0,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,2,1,1,1,1,0,...,0,0,2023-06-24,20:00,2023,3,2023-06-24-20:00|Florida Panthers vs Edmonton ...,2.45,4.000,2.45
1,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,5,1,1,0,2,0,...,0,0,2023-06-21,20:00,2023,6,2023-06-21-20:00|Edmonton Oilers vs Florida Pa...,2.35,4.000,2.55
2,https://www.flashscore.com/match/hockey/edmont...,Reg,Florida Panthers,Edmonton Oilers,3,5,0,1,2,3,...,0,0,2023-06-18,20:00,2023,8,2023-06-18-20:00|Florida Panthers vs Edmonton ...,2.00,4.000,3.10
3,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,8,1,3,1,3,0,...,0,0,2023-06-15,20:00,2023,9,2023-06-15-20:00|Edmonton Oilers vs Florida Pa...,2.30,4.000,2.55
4,https://www.flashscore.com/match/hockey/edmont...,Reg,Edmonton Oilers,Florida Panthers,3,4,0,1,1,3,...,0,0,2023-06-13,20:00,2023,7,2023-06-13-20:00|Edmonton Oilers vs Florida Pa...,2.15,4.000,2.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3344,https://www.flashscore.com/match/hockey/minnes...,AOT,Winnipeg Jets,Minnesota Wild,2,3,2,0,0,1,...,0,0,2025-09-21,17:00,2025,5,2025-09-21-17:00|Winnipeg Jets vs Minnesota Wild,2.20,4.000,2.45
3345,https://www.flashscore.com/match/hockey/florid...,Reg,Nashville Predators,Florida Panthers,5,0,1,0,2,0,...,0,0,2025-09-21,15:00,2025,5,2025-09-21-15:00|Nashville Predators vs Florid...,2.20,4.000,2.45
3346,https://www.flashscore.com/match/hockey/ottawa...,Reg,Ottawa Senators,Toronto Maple Leafs,3,4,0,3,2,1,...,0,0,2025-09-21,15:00,2025,7,2025-09-21-15:00|Ottawa Senators vs Toronto Ma...,1.44,5.250,4.50
3347,https://www.flashscore.com/match/hockey/new-je...,Reg,New Jersey Devils,New York Rangers,3,5,1,1,1,4,...,0,0,2025-09-21,13:00,2025,8,2025-09-21-13:00|New Jersey Devils vs New York...,2.20,4.000,2.45


In [18]:
sorted(final_data['X_Odds'].unique())

[np.float64(3.775),
 np.float64(3.8),
 np.float64(3.825),
 np.float64(3.8499999999999996),
 np.float64(3.875),
 np.float64(3.9),
 np.float64(3.925),
 np.float64(3.9499999999999997),
 np.float64(3.95),
 np.float64(3.975),
 np.float64(4.0),
 np.float64(4.05),
 np.float64(4.0649999999999995),
 np.float64(4.1),
 np.float64(4.125),
 np.float64(4.15),
 np.float64(4.165),
 np.float64(4.175),
 np.float64(4.2),
 np.float64(4.215),
 np.float64(4.225),
 np.float64(4.25),
 np.float64(4.265000000000001),
 np.float64(4.275),
 np.float64(4.29),
 np.float64(4.3),
 np.float64(4.300000000000001),
 np.float64(4.325),
 np.float64(4.33),
 np.float64(4.35),
 np.float64(4.375),
 np.float64(4.4),
 np.float64(4.415),
 np.float64(4.425),
 np.float64(4.45),
 np.float64(4.475),
 np.float64(4.5),
 np.float64(4.55),
 np.float64(4.575),
 np.float64(4.6),
 np.float64(4.625),
 np.float64(4.675),
 np.float64(4.75),
 np.float64(4.8),
 np.float64(4.825),
 np.float64(4.875),
 np.float64(4.925),
 np.float64(5.0),
 np.float